# Amazon Recommendation System - Lab

## Introduction

Now that you've gotten an introduction to collaborative filtering and recommendation systems, it's time to put your skills to test and build a recommendation system for a real world dataset! For this lab, you'll be using a dataset regarding the book reviews on the Amazon marketplace. While the previous lesson focused on user-based recommendation systems, you'll apply a parallel process for an item-based recommendation system to recommend similar books at the bottom of the product page.

## Objectives

In this lab you will: 

- Use graph-based similarity metrics to create a collaborative filtering recommender system

## Load the Dataset

In [1]:
import pandas as pd
import networkx as nx
G = nx.Graph()

df = pd.read_csv('books_data.edgelist', names=['source', 'target', 'weight'], delimiter=' ')
df.head()

,source,target,weight
0,0827229534,0804215715,0.7
1,0827229534,156101074X,0.5
2,0827229534,0687023955,0.8
3,0827229534,0687074231,0.8
4,0827229534,082721619X,0.7


## Load the Metadata 

Import the metadata available in the file `'books_meta.txt'` (note it is `'\t'` seperated). 

In [2]:
# Your code here
meta = pd.read_csv('books_meta.txt', sep='\t')
meta.head()

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
0,1,0827229534,Patterns of Preaching: A Sermon Sampler,clergi sermon subject religion preach spiritu ...,Book,396585,2,5.0,8,0.80
1,2,0738700797,Candlemas: Feast of Flames,subject witchcraft earth religion spiritu base...,Book,168596,12,4.5,9,0.85
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
3,4,0842328327,Life Application Bible Commentary: 1 and 2 Tim...,spiritu translat commentari christian book gui...,Book,631289,1,4.0,6,0.79
4,5,1577943082,Prayers That Avail Much for Business: Executive,subject religion spiritu busi christian live w...,Book,455160,0,0.0,4,1.00


## Select Books to Test Your Recommender On

Select a small subset of books that you are interested in generating recommendations for. 

In [3]:
# Your code here
GOT = meta[meta.Title.str.contains('Thrones')]
GOT

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
59750,83450,0553103547,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,16330,1191,4.5,4,0.67
130560,182190,1572701293,"Thrones, Dominations",literatur seri tape book format fiction genera...,Book,395606,61,3.5,4,0.90
183820,256164,0553573403,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,969,1196,4.5,7,0.48
261763,362549,0553381687,"A Game of Thrones (A Song of Ice and Fire, Boo...",general subject martin author epic z seri fant...,Book,11463,1196,4.5,4,0.67
331188,457079,0312968302,"Thrones, Dominations (A Lord Wimsey Mystery)",general subject british author sayer dorothi l...,Book,68918,61,3.5,7,0.81


## Generate Recommendations for a Few Books of Choice

The `'books_data.edgelist'` has conveniently already calculated the distance between items for you. Given this preprocessed data, it's time to employ collaborative filtering to generate recommendations! Generate the top 10 recommendations for each book in the subset you chose. Be sure to print the book name that you are generating recommendations for as well as the name of the books being recommended. 

In [4]:
# Your code here
rec_dict = {}
id_name_dict = dict(zip(meta.ASIN, meta.Title))
for row in GOT.index:
    book_id = GOT.ASIN[row]
    book_name = id_name_dict[book_id]
    most_similar = df[(df.source == book_id)
                      | (df.target == book_id)
                     ].sort_values(by='weight', ascending=False).head(10)
    most_similar['source_name'] = most_similar['source'].map(id_name_dict)
    most_similar['target_name'] = most_similar['target'].map(id_name_dict)
    recommendations = []
    for row in most_similar.index:
        if most_similar.source[row] == book_id:
            recommendations.append((most_similar.target_name[row], most_similar.weight[row]))
        else:
            recommendations.append((most_similar.source_name[row], most_similar.weight[row]))
    rec_dict[book_name] = recommendations
    print('Recommendations for:', book_name)
    for r in recommendations:
        print(r)
    print('\n\n')

Recommendations for: A Game of Thrones (A Song of Ice and Fire, Book 1)
('A Clash of Kings (A Song of Ice and Fire, Book 2)', 1.0)
('A Feast for Crows (A Song of Ice and Fire, Book 4)', 0.92)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 0.85)
("Assassin's Apprentice (The Farseer Trilogy, Book 1)", 0.56)



Recommendations for: Thrones, Dominations
('Have His Carcase', 0.59)
('The Nine Tailors', 0.58)
('Strong Poison', 0.55)
("Busman's Honeymoon", 0.55)



Recommendations for: A Game of Thrones (A Song of Ice and Fire, Book 1)
('A Storm of Swords : Book Three of A Song of Ice and Fire (A Song of Ice and Fire, Book 3)', 1.0)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 1.0)
('A Clash of Kings (A Song of Ice and Fire, Book 2)', 1.0)
('A Feast for Crows (A Song of Ice and Fire, Book 4)', 0.92)
('A Storm of Swords (A Song of Ice and Fire, Book 3)', 0.85)
("Assassin's Apprentice (The Farseer Trilogy, Book 1)", 0.56)
('The Fourth Tower of Inverness', 0.24)



Recommendatio

In [6]:
WWII = meta[meta.Title.str.contains('World War II')][:5]
WWII

,Id,ASIN,Title,Categories,Group,SalesRank,TotalReviews,AvgRating,DegreeCentrality,ClusteringCoeff
2,3,0486287785,World War II Allied Fighter Planes Trading Cards,general hobbi subject craft home garden book,Book,1270652,1,5.0,0,0.00
13,16,0195110382,War at Sea: A Naval History of World War II,general militari naval ii subject war world bo...,Book,631564,10,4.5,3,0.58
651,896,0395913756,Fighting For Honor : Japanese Americans and Wo...,general subject literatur unit asian state wor...,Book,144238,1,4.0,2,0.00
2817,3916,0964586711,PC Patrol Craft of World War II,general militari naval ii subject war world bo...,Book,1137130,3,5.0,3,0.58
4043,5675,0465001637,"Not Like Us: How Europeans Have Loved, Hated a...",general nonfict subject america cultur unit po...,Book,102107,2,5.0,8,0.00


In [7]:
rec_dict = {}
id_name_dict = dict(zip(meta.ASIN, meta.Title))
for row in WWII.index:
    book_id = WWII.ASIN[row]
    book_name = id_name_dict[book_id]
    most_similar = df[(df.source == book_id)
                      | (df.target == book_id)
                     ].sort_values(by='weight', ascending=False).head(10)
    most_similar['source_name'] = most_similar['source'].map(id_name_dict)
    most_similar['target_name'] = most_similar['target'].map(id_name_dict)
    recommendations = []
    for row in most_similar.index:
        if most_similar.source[row] == book_id:
            recommendations.append((most_similar.target_name[row], most_similar.weight[row]))
        else:
            recommendations.append((most_similar.source_name[row], most_similar.weight[row]))
    rec_dict[book_name] = recommendations
    print('Recommendations for:', book_name)
    for r in recommendations:
        print(r)
    print('\n\n')

Recommendations for: World War II Allied Fighter Planes Trading Cards



Recommendations for: War at Sea: A Naval History of World War II
('How They Won the War in the Pacific: Nimitz and His Admirals', 1.0)
('Touched With Fire: The Land War in the South Pacific', 0.89)
("Midway: The Battle that Doomed Japan, the Japanese Navy's Story", 0.82)



Recommendations for: Fighting For Honor : Japanese Americans and World War II
('Remembering Manzanar: Life in a Japanese Relocation Camp', 0.52)
('Runaway Girl : The Artist Louise Bourgeois (Bccb Blue Ribbon Nonfiction Book Award (Awards))', 0.18)



Recommendations for: PC Patrol Craft of World War II
('Splinter Fleet: The Wooden Subchasers of World War II', 0.9)
('U.S. Destroyers: An Illustrated Design History, Revised Edition', 0.41)
('U.S. Cruisers: An Illustrated Design History', 0.36)



Recommendations for: Not Like Us: How Europeans Have Loved, Hated and Transformed American Culture Since World War II
('Seducing the French: The Dilemma 

## Summary

Well done! In this lab, you effectively created a recommendation system for a real world dataset!